In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'IlyaGusev/saiga_llama3_8b'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # load_in_8bit=True,
    torch_dtype = torch.bfloat16,
    device_map = "auto"
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [3]:
from transformers import GenerationConfig
generation_config = GenerationConfig.from_pretrained(model_id)
print(generation_config)

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_new_tokens": 1536,
  "pad_token_id": 128000,
  "repetition_penalty": 1.12,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}



In [4]:
type(generation_config)

transformers.generation.configuration_utils.GenerationConfig

In [5]:
generation_config.max_new_tokens = 300
generation_config.temperature = 0.2
generation_config.top_k= 10

In [6]:
def get_llm_answer(query, chunks_join):
    user_prompt = '''Используй только следующий контекст, чтобы очень кратко ответить на вопрос в конце.
    Не пытайся выдумывать ответ.
    Контекст:
    ===========
    {chunks_join}
    ===========
    Вопрос:
    ===========
    {query}'''.format(chunks_join=chunks_join, query=query)
    
    SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."
    RESPONSE_TEMPLATE = "<|im_start|>assistant\n"
    
    prompt = f'''<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n<|im_start|>user\n{user_prompt}<|im_end|>\n{RESPONSE_TEMPLATE}'''
    
    def generate(model, tokenizer, prompt):
        data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
        data = {k: v.to(model.device) for k, v in data.items()}
        output_ids = model.generate(
            **data,
            generation_config=generation_config
        )[0]
        output_ids = output_ids[len(data["input_ids"][0]) :]
        output = tokenizer.decode(output_ids, skip_special_tokens=True)
        return output.strip()
    
    response = generate(model, tokenizer, prompt)
    
    return response

## inference

In [7]:
import gradio as gr

In [8]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [9]:
# embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru")

In [10]:
# from utils.reranker import real_rerank
from utils.vector_data import search_best_from_structured, search_best_from_unstructured



/home/yagor/anaconda3/envs/torch/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
query = 'Как и куда загружать выписки по филиалу акционерного общества «РУСБУРМАШ» в Республике Казахстан в системе XС ERP X.X?'

In [12]:
structured_res = search_best_from_structured(query)
structured_res

ffghnfgnfg


Document(page_content='Добрый день. Подскажите пожалуйста,  где сейчас в системе XС ERP X.X найти счета-фактуры выданные на аванс ?', metadata={'Код': 'IM22439872', 'Категория': 'запрос на обслуживание', 'Время регистрации': '16.01.2024 11:03:21', 'Рабочая группа': '1с erp 2.0 1-я линия', 'Краткое описание': 'Счет-фактура на аванс', 'Решение': 'Добрый день Оксана Владимировна! Перейти к счетам-фактурам выданным на аванс можно так: Финансовый результат и контроллинг – НДС – Счета-фактуры и таможенные документы. Здесь нажать на ссылку «Настроить», в открывшемся окне нажать «Снять пометки», отметить галочкой только «Счет-фактура выданный (аванс)», нажать «Выбрать». Далее можно сделать отбор по организации и/или контрагенту, а также задать необходимый период. Настроенный журнал можно сохранить в избранном, нажав на звездочку.', 'Аналитика 1': 'НДС', 'Аналитика 2': 'Учет НДС', 'Аналитика 3': 'Счета-фактуры'})

In [13]:
structured_res

Document(page_content='Добрый день. Подскажите пожалуйста,  где сейчас в системе XС ERP X.X найти счета-фактуры выданные на аванс ?', metadata={'Код': 'IM22439872', 'Категория': 'запрос на обслуживание', 'Время регистрации': '16.01.2024 11:03:21', 'Рабочая группа': '1с erp 2.0 1-я линия', 'Краткое описание': 'Счет-фактура на аванс', 'Решение': 'Добрый день Оксана Владимировна! Перейти к счетам-фактурам выданным на аванс можно так: Финансовый результат и контроллинг – НДС – Счета-фактуры и таможенные документы. Здесь нажать на ссылку «Настроить», в открывшемся окне нажать «Снять пометки», отметить галочкой только «Счет-фактура выданный (аванс)», нажать «Выбрать». Далее можно сделать отбор по организации и/или контрагенту, а также задать необходимый период. Настроенный журнал можно сохранить в избранном, нажав на звездочку.', 'Аналитика 1': 'НДС', 'Аналитика 2': 'Учет НДС', 'Аналитика 3': 'Счета-фактуры'})

In [15]:
unstructured_res = search_best_from_unstructured(query)
unstructured_res

Document(page_content='при настройке видов сертификатов.  \n\uf0b7 Счета учета расчет ов с контрагентами, настраиваемые с использованием \nэлементов списка  Группы настроек финансового учета расчетов, представлены в \nвиде групп в соответствии с их применением в операциях:  \no Расчетов с клиентами;  \no Расчетов с поставщиками;  \no Расчетов с комиссионерами;  \no Расчетов с комитентами;  \no Расчетов с кредиторами;  \no Расчетов с дебиторами;  \no Расчетов с лизингодателями.  \n\uf0b7 Производства  – задаются счета учета прямых производственных затрат \n(номенклатурные затраты на счетах 20, 23, 29) для подразделений в рамках \nкаждой организац ии. \n\uf0b7 Внеоборотных активов  – счета учета внеоборотных активов, настраиваемые с \nиспользованием элементов списка  Группы финансового учета внеоборотных \nактивов  при ведении в информационной базе учета внеоборотных активов по \nверсии 2.4.  \n\uf0b7 Прочих расходов  – счета у чета постатейных расходов, настраиваемые с \nиспользованием 

In [19]:

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    button = gr.Button("Отправить")
    clear = gr.ClearButton([msg, chatbot])
    def respond(input, history=None):
        res = ''
        excel_chunk = search_best_from_structured(input)
        if excel_chunk:
            print(excel_chunk)
            res += f'Решение аналогичного вопроса: {excel_chunk.metadata["Решение"]}'
            category = ' | '.join([f'{i}: {str(excel_chunk.metadata[i])}' for i in ["Аналитика 1", "Аналитика 2", "Аналитика 3"] ])
            res += f'\n {category}'
        
        pdf_chunk = search_best_from_unstructured(input)
        result = get_llm_answer(query, pdf_chunk.page_content).split('<|im_end|>')[0]
        page = pdf_chunk.metadata["source"].split("/")[-1].split("_")[-1].split(".")[0]
        file_name = pdf_chunk.metadata['source'].split('/')[-2] + ".pdf"
        info = f'Страница: {page}, Исходный документ: {file_name}'
        res += '\n' + f'Ответ LLM: {result}' + '\n' + info
        if history is None:
            history = []
        history.append((input, res))
        return '', history

    # def respond(message, chat_history):
    #     bot_message = random.choice(["How are you?", "I love you", "I'm very hungry"])
    #     chat_history.append((message, bot_message))
    #     return "", chat_history
    
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    button.click(respond, [msg, chatbot], [msg, chatbot])
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7865


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Running on public URL: https://6d7a49bdecdd74b248.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


ffghnfgnfg
page_content='Добрый день. Подскажите пожалуйста,  где сейчас в системе XС ERP X.X найти счета-фактуры выданные на аванс ?' metadata={'Код': 'IM22439872', 'Категория': 'запрос на обслуживание', 'Время регистрации': '16.01.2024 11:03:21', 'Рабочая группа': '1с erp 2.0 1-я линия', 'Краткое описание': 'Счет-фактура на аванс', 'Решение': 'Добрый день Оксана Владимировна! Перейти к счетам-фактурам выданным на аванс можно так: Финансовый результат и контроллинг – НДС – Счета-фактуры и таможенные документы. Здесь нажать на ссылку «Настроить», в открывшемся окне нажать «Снять пометки», отметить галочкой только «Счет-фактура выданный (аванс)», нажать «Выбрать». Далее можно сделать отбор по организации и/или контрагенту, а также задать необходимый период. Настроенный журнал можно сохранить в избранном, нажав на звездочку.', 'Аналитика 1': 'НДС', 'Аналитика 2': 'Учет НДС', 'Аналитика 3': 'Счета-фактуры'}
